#Databricks Platform Overview
This project aims to load the raw data from the wind turbines into Databricks, process & transform, and provide a platform for analytics on this data.

##Database Objects
We use a Unity Catalog enabled Databricks workspace, with a catalog created for this project `lakehouse_sbx`. Under this is a dedicated schema for the project called `cd_edw`. This is a managed schema located on a dedicated storage account container and will serve as a central location for all objects created as part of this project.

The project also follows the medallion architecture of Bronze -> Silver -> Gold where possible.

##Ingest Raw Data
Ingest raw CSV data from Azure Data Lake. This location is saved and exposed as a volume on Databricks Unity Catalog under the name `/Volumes/lakehouse_sbx/cd_edw/landing`.

We specify the schema to avoid issues in the future, but also include a `_rescued_data` column to catch any new fields that may appear in the future.

Since the source data is an append-only set of CSV files that is updated once daily, the following options were considered:
1. Batch loads on a trigger everyday after the files are updated to load all raw data into a Dataframe and overwrite the Bronze table.
1. Set up a Python method to track the rows that were read for each file, and every subsequent load would skip the previously loaded lines in the CSV and load only the new records (if any exist). 
    - This essentially decouples the integration layer with the data load layer, allowing the Databricks pipeline to be run whenever and at any frequency, regardless of when data is landed in the data lake.
    - Reduces overhead in the future as the data grows as data is always incrementally processed.
    - Caveat, this adds a bit of complexity to the pipeline, with an audit log Delta Table and reading from and writing to this table during each data load run.

###Questions
1. Do we have control over the ingest pattern to create new files that are date-partitioned instead of appending to a single/group of files? This could allow for using Autoloader, which handles checkpoints and processed files automatically enabling incremental loads.

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, TimestampType, DecimalType
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from datetime import timedelta
import pandas as pd
from delta.tables import DeltaTable

schema = StructType([
  StructField("timestamp", TimestampType(), True),
  StructField("turbine_id", IntegerType(), True),
  StructField("wind_speed", DecimalType(6, 2), True),
  StructField("wind_direction", DecimalType(6, 2), True),
  StructField("power_output", DecimalType(6, 2), True)
])

In [0]:
spark.sql("""
CREATE TABLE IF NOT EXISTS lakehouse_sbx.cd_edw.edw_turbine_ingest_audit_log (
    file_name STRING,
    file_path STRING,
    last_processed_position LONG,
    process_time TIMESTAMP
) USING DELTA
""")

In [0]:
def load_csv_incremental(schema, file_path):
    file_path_list = dbutils.fs.ls(file_path)

    final_df = spark.createDataFrame([], schema=schema)

    for file in file_path_list:
        last_position = spark.sql(f"""
                SELECT COALESCE(MAX(last_processed_position), 0) as pos
                FROM lakehouse_sbx.cd_edw.edw_turbine_ingest_audit_log
                WHERE file_name = '{file.name}'
            """).collect()[0]['pos']
        
        temp_df = (spark.read.format("csv") \
            .option("header", True) \
            .option("mode", "PERMISSIVE") \
            .option("rescuedDataColumn", "_rescued_data") \
            .option("skipRows", last_position)
            .schema(schema) \
            .load(f"{file_path}{file.name}") \
            .select(
                "*"
                , col("_metadata.file_path").alias("raw_file_path")
                , element_at(split("_metadata.file_path", '/'), array_size(split("_metadata.file_path", '/'))).alias("raw_file_name")
                , from_utc_timestamp(current_timestamp(), 'GMT').alias("bronze_processing_time")
            ))
                
        if temp_df.count() > 0:
            temp_df.write \
                .option("overwriteSchema", "true") \
                .saveAsTable("`lakehouse_sbx`.`cd_edw`.`brnz_turbine_data`",mode="append")

            new_position = last_position + temp_df.count()
            spark.sql(f"""
                INSERT INTO lakehouse_sbx.cd_edw.edw_turbine_ingest_audit_log 
                VALUES (
                    '{file.name}'
                    , '{file_path}'
                    , '{new_position}'
                    , current_timestamp()
                )
            """)




##Create Transformation Tables for Silver Layer
Create all tables required for cleaning and transforming Turbine data. Performs the following steps:
1. Create temporary dataframe with all 24h for the last 30d at the time of processing. This will be used to impute any missing values.
1. Join this against the Bronze table to get a "master" list of all datapoints over the last 30 days, with readings as NULLs for where they are missing.
1. Impute these missing values from the previous known readings. A simple `last` function is used to forward fill values.

In [0]:
def mergeIntoDeltaTable(df, table_exists, table_name):
    if not table_exists:
        df.write \
            .mode("overwrite") \
            .clusterBy("turbine_id", "date") \
            .saveAsTable(table_name)
    else:
        target_table = DeltaTable.forName(spark, table_name)
        (
            target_table.alias("target")
            .merge(
                df.alias("source"),
                "target.timestamp = source.timestamp AND target.turbine_id = source.turbine_id"
            )
            .whenMatchedUpdateAll()
            .whenNotMatchedInsertAll()
            .execute()
        )

def imputeMissingData():
    data_df = spark.read.format("delta") \
                        .table("lakehouse_sbx.cd_edw.brnz_turbine_data") \
                        .select("timestamp", "turbine_id", "wind_speed", "wind_direction", "power_output")

    impute_table_name = "lakehouse_sbx.cd_edw.slvr_turbine_data_01"
    min_date, max_date = data_df.select(min("timestamp"), max("timestamp")).first()
    table_exists = spark.catalog.tableExists(impute_table_name)

    if table_exists:
        min_date = max_date - timedelta(days=10)

    hourly_range = pd.date_range(start=min_date, end=max_date, freq='H')

    all_hours = spark.createDataFrame(
        [(ts.to_pydatetime(),) for ts in hourly_range],
        ["timestamp"]
    )

    all_turbines = data_df.select("turbine_id").distinct()

    complete_index = all_hours.crossJoin(all_turbines)

    window_spec = Window.partitionBy("turbine_id").orderBy("timestamp").rowsBetween(-24, 0)

    final_filled_df = complete_index.join(broadcast(data_df), ["timestamp", "turbine_id"], "left") \
        .withColumn("wind_speed", coalesce("wind_speed", last("wind_speed", ignorenulls=True).over(window_spec))) \
        .withColumn("wind_direction", coalesce("wind_direction", last("wind_direction", ignorenulls=True).over(window_spec))) \
        .withColumn("power_output", coalesce("power_output", last("power_output", ignorenulls=True).over(window_spec))) \
        .withColumn("date", to_date(col("timestamp"))) \
        .withColumn("month", month(col("timestamp"))) \

    final_filled_df = final_filled_df.repartition("turbine_id", "date")

    mergeIntoDeltaTable(final_filled_df, table_exists, impute_table_name)

##Calculate statistics and transformations
Use the cleaned and filled in data from above and calculate 24h and 30d averages and standard deviations.

###Assumptions
The following assumptions are made here:
1. Anomalies are calculated over a sliding 24h window, as well as a calendar day window. These are created as flags that can be filtered on.
1. For each window, the averages and standard deviations are calculated, and anomalies are where the reading is 2x stddev above the mean.

In [0]:
# from pyspark.sql.window import Window

def calculateStats():
    stats_table_name = "lakehouse_sbx.cd_edw.slvr_turbine_data_02"
    table_exists = spark.catalog.tableExists(stats_table_name)
    data_df = spark.read.format("delta") \
                        .table("lakehouse_sbx.cd_edw.slvr_turbine_data_01") \
                        .select("timestamp", "date", "month", "turbine_id", "wind_speed", "wind_direction", "power_output")

    window_daily = Window.partitionBy("turbine_id", "date")
    window_24h = Window.partitionBy("turbine_id").orderBy("timestamp").rowsBetween(-24, 0)
    window_monthly = Window.partitionBy("turbine_id", "month")

    stats_df = data_df \
                .withColumn("day_avg_output", avg("power_output").over(window_daily).cast("decimal(6,2)")) \
                .withColumn("day_stddev_output", stddev("power_output").over(window_daily).cast("decimal(7,3)")) \
                .withColumn("24h_avg_output", avg("power_output").over(window_24h).cast("decimal(6,2)")) \
                .withColumn("24h_stddev_output", stddev("power_output").over(window_24h).cast("decimal(7,3)")) \
                .withColumn("is_24h_anomaly", coalesce((col("power_output") > (col("24h_avg_output") + (2 * col("24h_stddev_output")))), lit(False))) \
                .withColumn("is_day_anomaly", coalesce((col("power_output") > (col("day_avg_output") + (2 * col("day_stddev_output")))), lit(False))) \
                .withColumn("silver_processing_time", from_utc_timestamp(current_timestamp(), 'GMT'))

    stats_df = stats_df.repartition("turbine_id", "date")

    # stats_df.sort("timestamp","turbine_id") \
    #     .where(stats_df.turbine_id == 1) \
    #     .display()

    mergeIntoDeltaTable(stats_df, table_exists, stats_table_name)

##Create Aggregated Tables
Use the silver tables from above to create and refresh daily and monthly statistics tables. These provide MIN(), MAX(), and AVG() for all turbines over daily and monthly periods.

These tables serve as the "Gold" layer and can be used for reporting and analytics.

In [0]:
def dailyAggregates(data_df):
    daily_df = data_df.groupBy("turbine_id", "date") \
                    .agg(
                        min("power_output").alias("min_output"),
                        max("power_output").alias("max_output"),
                        avg("power_output").alias("avg_output")
                    ) \

    daily_df.write \
            .mode("overwrite") \
            .clusterBy("turbine_id", "date") \
            .saveAsTable("lakehouse_sbx.cd_edw.turbine_stats_daily")

In [0]:
def monthlyAggregates(data_df):
    monthly_df = data_df.groupBy("turbine_id", "month") \
                    .agg(
                        min("power_output").alias("min_output"),
                        max("power_output").alias("max_output"),
                        avg("power_output").alias("avg_output")
                    ) 

    monthly_df.write \
            .mode("overwrite") \
            .clusterBy("turbine_id", "month") \
            .saveAsTable("lakehouse_sbx.cd_edw.turbine_stats_monthly")

In [0]:
def anomaliesTable(data_df):
    anomalies_df = data_df.where(data_df.is_24h_anomaly == True)

    return(anomalies_df.write \
            .mode("overwrite") \
            .clusterBy("turbine_id", "month") \
            .saveAsTable("lakehouse_sbx.cd_edw.turbine_24h_anomalies"))


##Simple Orchestrator
Bring together and execute all functions defined above in one location. This notebook can be executed by a Job/Workflow on Databricks.

In [0]:
if __name__ == "__main__":
    load_csv = load_csv_incremental(schema, '/Volumes/lakehouse_sbx/cd_edw/landing/cd_raw_data/')
    impute_data = imputeMissingData()
    calc_stats = calculateStats()
    data_df = spark.table("lakehouse_sbx.cd_edw.slvr_turbine_data_02")
    daily_agg = dailyAggregates(data_df)
    monthly_agg = monthlyAggregates(data_df)
    anomalies_table = anomaliesTable(data_df)
